In [4]:
import os  # <--- Add this line
from langgraph.graph import StateGraph ,START, END
from typing import TypedDict

In [ ]:
class batsmanState(TypedDict):
    #input state
    balls:int
    runs:int
    sixes:int
    fours:int

    #output state
    sr:float
    bpb:float
    boundary_percent:float
    

In [8]:
def calculate_sr(state:batsmanState)->batsmanState:
    runs=state['runs']

In [9]:
def calculate_bpb(state:batsmanState)->batsmanState:
    runs=state['runs']

In [10]:
def calculate_boundaryPer(state:batsmanState)->batsmanState:
    runs=state['runs']

In [ ]:
graph = StateGraph(batsmanState)

#nodes
graph.add_edge('calculate_sr',calculate_sr)
graph.add_edge('calculate_bpb',calculate_bpb)
graph.add_edge('calculate_boundaryPer',calculate_boundaryPer)
graph.add_edge('summary',summary)

#parallel flow 
graph.add_edge(START,'calculate_sr')
graph.add_edge(START,'calculate_bpb')
graph.add_edge(START,'calculate_boundaryPer')

graph.add_edge('calculate_bpb','summary')
graph.add_edge('calculate_sr','summary')
graph.add_edge('calculate_boundaryPer','summary')

graph.add_edge('summary',END)

#6.complie workflow
workflow=graph.compile()
